# 试卷手写笔迹擦除 <h2 align="right"> -- 百度网盘AI大赛：手写文字擦除第5名方案</h2>
![HDWkyF.png](http://bj.bcebos.com/v1/ai-studio-match/file/578e4fbe963f44358e77652ceb3b58a4883ad9a44395447aa239a07d33c1dc40?authorization=bce-auth-v1%2F0ef6765c1e494918bc0d4c3ca3e5c6d1%2F2021-12-05T16%3A50%3A15Z%2F-1%2F%2F44bf5a82a175a1cbef3cdf1655b36a15b67b4cd4a937dbf4f2b7b26578d9d591)

## 一、简介
### 1. 项目介绍
本项目基于 SegFormer 语义分割模型，识别试卷中的手写笔迹来生成去除掩码，实现 AI 笔迹橡皮擦，擦除试卷中的手写笔迹。
### 2. 项目背景
本项目源于百度网盘的图像处理挑战赛，手写文字擦除任务。
百度网盘AI大赛——图像处理挑战赛是百度网盘开放平台 面向AI开发者和爱好者发起的计算机视觉领域挑战赛。AI时代已到来，百度网盘与百度飞桨AI Studio强强联合，旨在基于个人云存储的生态能力开放，通过比赛机制，鼓励选手结合当下前沿的计算机视觉技术与图像处理技术，完成模型设计搭建与训练优化，产出基于飞桨框架的开源模型方案，为中国开源生态建设贡献一份力量。
本次图像处理挑战赛以线上比赛的形式进行，参赛选手需要在规定时间内，完成模型设计搭建与训练优化，并提交基于评测数据集产出的结果文件
### 3. 赛题分析
赛题描述：选手需要建立模型，对比赛给定的带有手写痕迹的试卷图片进行处理，擦除相关的笔，还原图片原本的样子，并提交模型输出的结果图片。
思路分析：试卷中存在的痕迹可以视为印刷和手写两类，手写字体的笔迹往往不够规整，在整体结构上不如印刷体的分明，因此我们可以将手写体和印刷体视为两种不同结构的语义类别，用语义分割模型对其处理，获得手写笔记的分割mask。但是大多数语义分割模型往往生成较为粗糙的 mask，我们需要对模型的 decoder 进行一定改动设计。

### 赛题分析

- **赛题描述**：选手需要建立模型，对比赛给定的带有手写痕迹的试卷图片进行处理，擦除相关的笔，还原图片原本的样子，并提交模型输出的结果图片。
- **思路分析**：试卷中存在的痕迹可以视为印刷和手写两类，手写字体的笔迹往往不够规整，在整体结构上不如印刷体的分明，因此我们可以将手写体和印刷体视为两种不同结构的语义类别，用语义分割模型对其处理，获得手写笔记的分割mask。但是大多数语义分割模型往往生成较为粗糙的 mask，我们需要对模型的 decoder 进行一定改动设计。

## 二、数据集介绍及处理

本次比赛最新发布的数据集中，所有的图像数据均由真实场景采集得到，再通过技术手段进行相应处理，生成可用的脱敏数据集。该任务为image-to-image的形式，因此源数据和GT数据均以图片的形式来提供。其中训练集有1000个样本，测试集A榜和B榜各200个样本训练数据集。训练数据集结构如下：

```
|- root  
    |- images
    |- gts
```

1. images 手写文字的源图像数据，gts 为无手写文字的真值数据（仅有训练集数据提供gts ，A榜测试集、B榜测试集数据均不提供gts）；
2. images 与 gts 中的图片根据图片名称一一对应。
   
   ![qi0q3Q.png](https://s1.ax1x.com/2022/03/18/qi0q3Q.png)

上图为训练集中的一对样本，我们可以观察得到以下三点特征：

1. 需要去除的手写痕迹在结构上比较的精细，而且大小区别很大
2. 虽然存在需要大量去除的痕迹，但是与整张图片比较，区域占比相对还是比较小
3. 手写痕迹存与印刷痕迹存在耦合，这对于完整结构的识别与分割提出了挑战

### 数据集处理

依据我们的设计，数据集处理分为以下部分：

1. **Mask 生成**：由于我们采用语义分割框架，我们需要生成二分类掩码的真实值。
   - 我们通过对比去除文字后和去除文字前的图像，对于大于设定阈值的地方，我们认为是需要除去的部分，置为 1 值，其余置为 0 值，以此生成 Mask，如下图所示：
     
     ![qiNzKx.png](https://s1.ax1x.com/2022/03/18/qiNzKx.png)
2. **图像输入预处理**：输入的图像往往是矩形图像，为了保持横纵比，对于非正方形图像，我们填充图像边缘形成正方向图像作为输入，同时输入图像的大小一，且有些图像像素尺寸较大，因此我们统一下采样图片至 384 * 384。
3. **训练数据增强**：在训练时，我们对数据进行一定的增强，加入了随机水平反转和随机裁剪等操作，增强模型的泛化能力。

## 三、模型设计

我们把笔记擦除任务视为笔迹识别并生成掩码的任务，而这恰好是语义分割所擅长的，因此我们考虑采用基于语义分割的 SOTA 模型--Segformer 来进行文字擦除。

**SegFormer**: Encoder - Decoder 结构，Encoder 捕获充足的上下文语义信息，Decoder 进行上采样得到像素分割掩码（但原文设计中最终只得到原图 4x 下采样的语义分割结果，需要上采样回去到输入图像的尺寸）。Encoder 由Transformer Block 组成，形成结构化特征表征；Decoder 由 MLP 组成，聚合不同尺度信息。整体框架如下所作：

<img src="https://s4.ax1x.com/2022/02/13/HDWVeJ.png" alt="HDWVeJ.png" style="zoom:75%;" />

#### SegFormer 设计的特点：

1. **Efficient Self-Attn**：通过将空间局部信息转化为通道信息，高效自注意力层将原始自注意力层 $O(N^2)$ 的计算复杂度降为了 $O(\frac{N^2}{R})$ ，同时通过这一操作还实现了全局--局部信息的交互，强化了信息提取能力。
   
   <img src="https://s1.ax1x.com/2022/03/18/qFFPvq.png" alt="qFFPvq.png" style="zoom:50%;" />
2. **Mix-FFN**：SegFormer 放弃了 Transformer 原始的位置编码，利用 MLP 和 $3 \times 3$ 卷积编码来实现局部位置信息的感知，从而能够更加灵活适应多种分辨率图像的输入，捕捉不同尺度的信息。
   
   <img src="https://s1.ax1x.com/2022/03/18/qFkUTU.png" alt="qFkUTU.png" style="zoom:50%;" />
3. **Lightweight All-MLP Decoder**：SegFormer 的 Decoder 仅由 MLP 和无参数上采样层组成，作者提出由于编码器中自注意力层的存在，在解码器中无需再设计复杂的模块（如Dilated Convolution 和 ASPP 设计）来扩大模型有效感受野，因此用 MLP 层进行信息融合即可。当然这也有一定弊端，无参数上采样层无法很好利用局部信息来插值，导致分割结果的边缘可能不够精细化。

*详见 [SegFormer: Simple and Eicient
Design for Semantic Segmentation with Transformers](https://arxiv.org/abs/2105.15203)


### 模型改进

试卷中的文字一般是精细化的结构，而 SegFormer 产生的掩码相对粗糙，不利于去除手写的文字痕迹，我们考虑采用可学习的转置卷积，通过局部区域特征上采样，不断补充精细结构来生成掩码结果，相较于无参数的上采样（如双线性插值），能够更好利用局部信息进行更高阶上采样。
![HDWuJx.png](https://s4.ax1x.com/2022/02/13/HDWuJx.png)

### 损失函数设计

一般的语义分割任务中的对象往往大尺寸的结构占主体，因此交叉熵损失就足够解决问题；但是手写痕迹的细化结构，会导致交叉熵损失效果不能起到最好效果，因此我们在交叉熵损失基础上还加入了 dice loss 针对细化结构进行有效惩罚。
$$Dice =\frac{2|X \bigcup Y|}{|X| + |Y|}$$
$$L_{dice} = 1 - Dice$$

其中 $X\bigcup Y$ 表示求取 X 和 Y 的并集。最终的损失函数如下：

$$L = L_{ce} + \lambda \times L_{dice}$$

$L_{ce}$ 为交叉熵损失，$\lambda$ 为平衡超参数，这里设置为0.5。

## 工程实践


### 1. 环境安装

In [1]:
!pip install -r work/requirement.txt

Looking in indexes: https://mirror.baidu.com/pypi/simple/, https://mirrors.aliyun.com/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.3/424.3 kB 6.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 16.7 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for visdom: filename=visdom-0.2.4-py3-none-any.whl size=1408188 sha256=0c37e8a473b51a8432637fa12d934db5a7c8a8f910923275ef7fb866d779629b
  Stored in directory: /home/aistudio/.cache/pip/wheels/0d/55/02/3143f973e037a3727c57d757d7106123057b42b50d51ce7ff9
Successfully built visdom

[notice] A new release of pip available: 22.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


### 2. 数据集生成

In [2]:
!ls data/data127971 # show the raw directory
!unzip data/data127971/dehw_train_dataset.zip -d data/ # extract the train data
!unzip data/data127971/dehw_testA_dataset.zip -d data/ # extract the test data
!python work/generate_files.py --trainData 1 --root data/dehw_train_dataset/images # generate the train list
!python work/generate_files.py --trainData 0 --root data/dehw_testA_dataset/images # generate the test list

  inflating: data/__MACOSX/dehw_train_dataset/gts/._dehw_train_00492.png  
  inflating: data/dehw_train_dataset/gts/dehw_train_00479.png  
  inflating: data/__MACOSX/dehw_train_dataset/gts/._dehw_train_00479.png  
  inflating: data/dehw_train_dataset/gts/dehw_train_01001.png  
  inflating: data/__MACOSX/dehw_train_dataset/gts/._dehw_train_01001.png  
  inflating: data/dehw_train_dataset/gts/dehw_train_01015.png  
  inflating: data/__MACOSX/dehw_train_dataset/gts/._dehw_train_01015.png  
  inflating: data/dehw_train_dataset/gts/dehw_train_00445.png  
  inflating: data/__MACOSX/dehw_train_dataset/gts/._dehw_train_00445.png  
  inflating: data/dehw_train_dataset/gts/dehw_train_00323.png  
  inflating: data/__MACOSX/dehw_train_dataset/gts/._dehw_train_00323.png  
  inflating: data/dehw_train_dataset/gts/dehw_train_01029.png  
  inflating: data/__MACOSX/dehw_train_dataset/gts/._dehw_train_01029.png  
  inflating: data/dehw_train_dataset/gts/dehw_train_00337.png  
  inflating: data/__MACOSX/

### 3. 训练模型

In [3]:
!python work/main.py\
    --train 1\
    --arch segformer_b2\
    --num_epochs 10\
    --batchSize 24\
    --dataRoot data/dehw_train_dataset\

W0629 15:49:00.337352  1410 device_context.cc:447] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 12.8, Runtime API Version: 10.1
W0629 15:49:00.342500  1410 device_context.cc:465] device: 0, cuDNN Version: 7.6.
2025-06-29 15:49:04 [INFO]	Loading pretrained model from https://bj.bcebos.com/paddleseg/dygraph/mix_vision_transformer_b2.tar.gz
Connecting to https://bj.bcebos.com/paddleseg/dygraph/mix_vision_transformer_b2.tar.gz
[==================================================] 100.00%
Uncompress mix_vision_transformer_b2.tar.gz
[==================================================] 100.00%
2025-06-29 15:49:07 [WARNING]	linear_c4.proj.weight is not in pretrained model
2025-06-29 15:49:07 [WARNING]	linear_c4.proj.bias is not in pretrained model
2025-06-29 15:49:07 [WARNING]	linear_c3.proj.weight is not in pretrained model
2025-06-29 15:49:07 [WARNING]	linear_c3.proj.bias is not in pretrained model
2025-06-29 15:49:07 [WARNING]	linear_c2.proj.weight is not in pretr

### 4. 结果生成

In [4]:
!python work/main.py\
    --train 0\
    --arch segformer_b2\
    --modelLog Log/segformer_b2/02131458.pdparams\
    --dataRoot data/dehw_testA_dataset\

W0629 16:05:05.434248 21262 device_context.cc:447] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 12.8, Runtime API Version: 10.1
W0629 16:05:05.439185 21262 device_context.cc:465] device: 0, cuDNN Version: 7.6.
2025-06-29 16:05:09 [INFO]	Loading pretrained model from Log/segformer_b2/02131458.pdparams
2025-06-29 16:05:09 [INFO]	There are 360/360 variables loaded into MySegFormer.
100%|███████████████████████████████████████████| 40/40 [00:33<00:00,  1.21it/s]


# 七、参考
1. Enze Xie, Wenhai Wang, Zhiding Yu, Anima Anandkumar, Jose M. Alvarez, and Ping Luo. SegFormer: Simple and Efficient Design for Semantic Segmentation with Transformers. *NeurIPS 2021.*
2. [语义分割之dice loss深度分析](https://zhuanlan.zhihu.com/p/269592183)
3. SegFormer 实现：https://github.com/NVlabs/SegFormer

